In [15]:
import numpy as np
import matplotlib.pyplot as plt

In [16]:
# Признаки типа float64 
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
#Целевая переменная типа float64
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [17]:
#функция масштабирования методом стандартизации 
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [18]:
#копируем признаки с помощью метода copy()(не очень понял зачем это делается)
X_st = X.copy()
#масштабирум значения
X_st[2, :] = calc_std_feat(X[2, :])

Домашняя работа

1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [19]:
#функционал ошибки. Область определения логарифмической функции x>0. Для того, чтобы нули не попадали np.log
#целесообразно прибавить к аргументу необольшое значение. Например +0.00000000000001 или меньше 1e-1000

def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred+0.00000000000001) + (1.0 - y) * np.log(1.0 - y_pred + 0.00000000000001))
    err = np.sum(err)
    return err

In [20]:
#функция возврата значения сигмоиды 

def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [21]:
#градиентный спуск 

def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [46]:
# err имеет очень маленькое значание, но с такими параметрами вероятно модель переобучена (судя по большим весам)

W = eval_model(X_st, y, iterations=10000, alpha=42)

1000 [-271.52790251  -38.26229776  -74.24304757  233.26443458] 0.004364214915315821
2000 [-274.30662092  -36.49037829  -75.7148352   232.75370324] 0.004047618633363658
3000 [-276.88812326  -34.84758582  -77.08081903  232.28385274] 0.0037748707493751726
4000 [-279.30233636  -33.31146235  -78.35817394  231.84476745] 0.0035363707327084283
5000 [-281.56973414  -31.86876093  -79.55782599  231.43241539] 0.0033260086275432197
6000 [-283.70704934  -30.50881996  -80.68863893  231.04372785] 0.003139099077906611
7000 [-285.72830324  -29.22271852  -81.75803471  230.67615156] 0.0029719433286592785
8000 [-287.64537361  -28.00290053  -82.77229996  230.32752536] 0.002821579617936955
9000 [-289.46841049  -26.84290903  -83.73680665  229.99600267] 0.0026856084344742868
10000 [-291.20615501  -25.73718325  -84.65618093  229.6799933 ] 0.002562065402016888


In [59]:
#вроде так нормально
W = eval_model(X_st, y, iterations=1000000, alpha=0.01)

100000 [-10.74417094  -1.38394305  -2.42070613   9.13379901] 0.2321437671191294
200000 [-15.42675299  -1.7813434   -3.82857757  12.91697903] 0.19343511937928798
300000 [-19.03147665  -2.10012227  -4.88442605  15.84668363] 0.17055165430889585
400000 [-21.9984015   -2.36612288  -5.74167338  18.25876214] 0.1550856771595887
500000 [-24.52626334  -2.59421122  -6.46565423  20.31251846] 0.143881308868302
600000 [-26.73370635  -2.79394547  -7.0941108   22.10416896] 0.13535239004099764
700000 [-28.69811567  -2.97178775  -7.65111645  23.6967112 ] 0.12860871579516717
800000 [-30.47261246  -3.13227528  -8.15292     25.1334636 ] 0.12311385430766472
900000 [-32.09507997  -3.27869631  -8.6109551   26.44534733] 0.11852642476064826
1000000 [-33.59338441  -3.41349945  -9.03354044  27.65512062] 0.11461933409504724


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [62]:
#необходимо найти скалярное произведение весов на признаки X и подставить в сигмоиду 
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(W@X)
    return y_pred_proba

In [63]:
calc_pred_proba(W, X_st)

array([3.76929812e-01, 1.43639783e-02, 9.99999995e-01, 1.10353190e-07,
       9.32591452e-01, 6.42813185e-02, 1.00000000e+00, 2.06620214e-02,
       6.05608740e-01, 9.99999456e-01])

4. Создайте функцию calc_pred, возвращающую предсказанный класс 
(на вход подаются W, который уже посчитан функцией eval_model и X, 
 на выходе - массив y_pred).

In [73]:
def calc_pred(W, X):
    thr = 0.5
    a = np.dot(W, X)
    pr = sigmoid(a)
    y_pred = np.where(pr > thr, 1, 0)
    return y_pred

In [76]:
calc_pred(W, X_st)

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [81]:
def ac_pr_re_f1(y, y_pred):
    TP = np.sum(np.logical_and(y_pred == 1, y == 1))
    TN = np.sum(np.logical_and(y_pred == 0, y == 0))
    FP = np.sum(np.logical_and(y_pred == 1, y == 0))
    FN = np.sum(np.logical_and(y_pred == 0, y == 1))
    ac = (TP + TN)/(TP + TN + FP + FN)
    pr = TP/(TP + FP)
    re = TP/(TP + FN)
    f1 = 2*pr*re/(pr + re)

    return ac, pr, re, f1

In [86]:
y_pred=calc_pred(W, X_st)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [87]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [88]:
ac_pr_re_f1(y, y_pred)

(1.0, 1.0, 1.0, 1.0)

6. Могла ли модель переобучиться? Почему?

Могла, потому что у нас мало входных данных, т.е непредсавительная выборка, а также ручной подбор порога разбияния на категории. Переобучение наблюдалось в решении задачи №2